In [22]:
import requests
import time
import json

import re
from html import unescape

headers = {"user-agent" : "Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36"}

def download(method, url, param=None, data=None, timeout=1 , maxretires=3):
    try : 
        resp = requests.request(method, url, params=param, data=data, headers=headers)
        resp.raise_for_status()
    except requests.exceptions.HTTPError as e :
        if 500 <= e.response.status_code < 600 and maxretires > 0:
            print(maxretires)
            time.sleep(timeout)
            resp = download(method, url, param, data, timeout, maxretires-1)
        else:
            print(e.response.status_code)
            print(e.response.reason)
    return resp

In [23]:
# beautifulSoup : Parsing library
!pip install beautifulsoup4
from bs4 import BeautifulSoup

# Crawling

- 봇(bot)
- 색인(indexing) 목적으로 쓴다.
- 데이터를 내 원하는 만큼 가져온다. 
- focused crawling 배운다.

In [3]:
# seed 주소 : http://example.webscraping.com/places/default/index/1
# a 태그에서 href 값을 가져올 것이다.

url = "http://example.webscraping.com/places/default/index"

html = download("get", url)

In [4]:
# 내부 링크.
dom = BeautifulSoup(html.text, "lxml")
[requests.compat.urljoin(url, _["href"])
 for _ in dom.find_all("a") if len(_["href"]) > 3]

['http://example.webscraping.com/places/default/user/register?_next=/places/default/index',
 'http://example.webscraping.com/places/default/user/login?_next=/places/default/index',
 'http://example.webscraping.com/places/default/index',
 'http://example.webscraping.com/places/default/search',
 'http://example.webscraping.com/places/default/view/Afghanistan-1',
 'http://example.webscraping.com/places/default/view/Aland-Islands-2',
 'http://example.webscraping.com/places/default/view/Albania-3',
 'http://example.webscraping.com/places/default/view/Algeria-4',
 'http://example.webscraping.com/places/default/view/American-Samoa-5',
 'http://example.webscraping.com/places/default/view/Andorra-6',
 'http://example.webscraping.com/places/default/view/Angola-7',
 'http://example.webscraping.com/places/default/view/Anguilla-8',
 'http://example.webscraping.com/places/default/view/Antarctica-9',
 'http://example.webscraping.com/places/default/view/Antigua-and-Barbuda-10',
 'http://example.webscr

In [9]:
def parseURL(seed):
    html = download("get", seed)
    dom = BeautifulSoup(html.text, 'lxml')
    
    return [requests.compat.urljoin(url, _["href"])
     for _ in dom.find_all("a") if len(_["href"]) > 3]

In [ ]:
# return 한 값을 큐에 넣는다.
# 알고리즘 URL-seen 과정 진행중...

queue = list()
queue.append("http://example.webscraping.com/places/default/index")
# 다른 검색결과에 들어가려면 이 줄에 넣는 것. 
seen = list()

# 너무 빠르게 request 하지 않도록 slip을 넣는다.
while queue:
    baseURL = queue.pop(0) # 무조건 앞에 있는 게 나온다. 선입선출.
    seen.append(baseURL)
    time.sleep(1) # 슬립
    linkList = parseURL(baseURL) # 중복되는 URL이 생긴다.
    for link in linkList: # 중복되는 url 있으면 피할 것이다.
        if link not in queue and link not in seen:
            queue.append(link)
    print("Queue: {0}, Seen : {1}".format(len(queue), len(seen)))
#     queue.extend(linkList)

In [16]:
# href 있는지 없는지 추가
def parseURL(seed):
    html = download("get", seed)
    dom = BeautifulSoup(html.text, 'lxml')
    
    return [requests.compat.urljoin(url, _["href"])
     for _ in dom.find_all("a") if _.has_attr("href") and len(_["href"]) > 3]

In [24]:
url = "https://www.google.com/search?"

param = {
    "q":"박보영"
}
html = download("get", url, param)
dom = BeautifulSoup(html.text, "lxml")

In [26]:
[_.find_parent()["href"] for _ in dom.select(".LC20lb")]

['https://ko.wikipedia.org/wiki/%EB%B0%95%EB%B3%B4%EC%98%81',
 'https://ko.wikipedia.org/wiki/%EB%B0%95%EB%B3%B4%EC%98%81%EC%9D%98_%EC%9E%91%ED%92%88_%EB%AA%A9%EB%A1%9D',
 'https://namu.wiki/w/%EB%B0%95%EB%B3%B4%EC%98%81',
 'https://news.joins.com/article/22895953',
 'https://www.msn.com/ko-kr/entertainment/news/%EB%B0%95%EB%B3%B4%EC%98%81-tvn-%EB%93%9C%EB%9D%BC%EB%A7%88-%EC%96%B4%EB%B9%84%EC%8A%A4-%EC%97%AC%EC%A3%BC%EC%9D%B8%EA%B3%B5/ar-BBLZzVh',
 'https://www.msn.com/ko-kr/entertainment/tv/hd%EC%8A%A4%ED%86%A0%EB%A6%AC-%EC%9E%91%EC%9D%80-%EA%B1%B0%EC%9D%B8-%EB%B0%95%EB%B3%B4%EC%98%81%E2%80%A6%EA%B7%B8%EC%9D%98-%EC%97%B0%EA%B8%B0%EC%97%90-%EB%8C%80%ED%95%9C-%ED%9D%94%EC%A0%81%EB%93%A4/ar-BBUvL0V',
 'https://movie.daum.net/person/main?personId=105244',
 'https://twitter.com/hashtag/%EB%B0%95%EB%B3%B4%EC%98%81']

In [27]:
queue = list()
queue.extend([_.find_parent()["href"] for _ in dom.select(".LC20lb")])
seen = list()

while queue:
    baseURL = queue.pop(0) # 무조건 앞에 있는 게 나온다. 선입선출. pop(-1) 로 바꾸면 스택이 된다.
    seen.append(baseURL)
    time.sleep(1) # 슬립
    linkList = parseURL(baseURL) # 중복되는 URL이 생긴다.
    for link in linkList: # 중복되는 url 있으면 피할 것이다.
        if link not in queue and link not in seen:
            queue.append(link)
    print("Queue: {0}, Seen : {1}".format(len(queue), len(seen)))
#     queue.extend(linkList)

Queue: 864, Seen : 1
Queue: 1123, Seen : 2
Queue: 2092, Seen : 3
Queue: 2214, Seen : 4
Queue: 2248, Seen : 5
Queue: 2257, Seen : 6
Queue: 2300, Seen : 7
Queue: 2483, Seen : 8
Queue: 2520, Seen : 9
Queue: 2522, Seen : 10
404
Not Found
Queue: 2522, Seen : 11
404
Not Found
Queue: 2521, Seen : 12
404
Not Found
Queue: 2520, Seen : 13
404
Not Found
Queue: 2519, Seen : 14
404
Not Found
Queue: 2518, Seen : 15


KeyboardInterrupt: 

## 네이버 focused crawling 하기

In [42]:
url = "https://search.naver.com/search.naver"
param = {
    "query":"박보영"
}
html = download("get", url, param)
dom = BeautifulSoup(html.text, "lxml")

In [43]:
[_["href"] for _ in dom.select("a.sh_blog_title._sp_each_url._sp_each_title")]

['https://blog.naver.com/imagetech1?Redirect=Log&logNo=221530358950',
 'https://asovc7777.blog.me/221548676988',
 'http://hshlog.com/221559890110',
 'https://blog.naver.com/js2y86?Redirect=Log&logNo=221530397718',
 'https://blog.naver.com/dlqlwm14?Redirect=Log&logNo=221529807195']

In [31]:
requests.compat.urlparse("https://blog.naver.com/dlqlwm14?Redirect=Log&logNo=221529807195")

ParseResult(scheme='https', netloc='blog.naver.com', path='/dlqlwm14', params='', query='Redirect=Log&logNo=221529807195', fragment='')

In [32]:
def checkBlog(url):
    return requests.compat.urlparse(url)[1] == "blog.naver.com"

In [34]:
checkBlog("https://blog.naver.com/dlqlwm14?Redirect=Log&logNo=221529807195")

True

In [35]:
checkBlog("http://hshlog.com/221559890110")

False

In [44]:
[_["href"] for _ in dom.select("a.sh_blog_title._sp_each_url._sp_each_title")
if checkBlog(_["href"])]

['https://blog.naver.com/imagetech1?Redirect=Log&logNo=221530358950',
 'https://blog.naver.com/js2y86?Redirect=Log&logNo=221530397718',
 'https://blog.naver.com/dlqlwm14?Redirect=Log&logNo=221529807195']

In [45]:
def parseURL(seed):
    html = download("get", seed)
    dom = BeautifulSoup(html.text, 'lxml')
    
    if len(dom.select("#mainFrame")) < 1:
        return []
   
    seed = requests.compat.urljoin(seed, dom.select("#mainFrame")[0]["src"])
   
    html = download("get", seed)
    dom = BeautifulSoup(html.text, 'lxml')
    
    return [requests.compat.urljoin(seed, _["href"])
     for _ in dom.find_all("a") if _.has_attr("href") and len(_["href"]) > 3
           and checkBlog(requests.compat.urljoin(seed, _["href"]))]

In [46]:
queue = list()
queue.extend([_["href"] for _ in dom.select("a.sh_blog_title._sp_each_url._sp_each_title")
if checkBlog(_["href"])])
seen = list()

while queue:
    baseURL = queue.pop(0)
    seen.append(baseURL)
    time.sleep(1) # 슬립
    linkList = parseURL(baseURL)
    for link in linkList:
        if link not in queue and link not in seen:
            queue.append(link)
    print("Queue: {0}, Seen : {1}".format(len(queue), len(seen)))
#     queue.extend(linkList)

Queue: 17, Seen : 1
Queue: 32, Seen : 2
Queue: 48, Seen : 3
Queue: 47, Seen : 4
Queue: 46, Seen : 5
Queue: 45, Seen : 6
Queue: 44, Seen : 7
Queue: 43, Seen : 8
Queue: 42, Seen : 9
Queue: 41, Seen : 10
Queue: 40, Seen : 11
Queue: 39, Seen : 12
Queue: 38, Seen : 13
Queue: 37, Seen : 14
Queue: 64, Seen : 15
Queue: 63, Seen : 16
404
Not Found
Queue: 62, Seen : 17
Queue: 61, Seen : 18
Queue: 60, Seen : 19
Queue: 59, Seen : 20
Queue: 58, Seen : 21
Queue: 57, Seen : 22
Queue: 56, Seen : 23
Queue: 55, Seen : 24
Queue: 54, Seen : 25


KeyboardInterrupt: 

## 다음focused crawling 하기

In [90]:
url = "https://search.daum.net/search"
param = {
    "q":"박보영"
}
html = download("get", url, param)
dom = BeautifulSoup(html.text, "lxml")

In [96]:
[_["href"] for _ in dom.select("#blogColl .f_link_b")]

['http://adam24eve.tistory.com/858',
 'http://cscs0717.tistory.com/27',
 'http://listup.tistory.com/248',
 'http://34344541.tistory.com/43']

In [125]:
def checkBlog(url):
    return requests.compat.urlparse(url)[1].endswith("tistory.com")

In [126]:
re.search(".+?\.tistory.com", "http://adam24eve.tistory.com/858")

<re.Match object; span=(0, 28), match='http://adam24eve.tistory.com'>

In [110]:
[_["href"] for _ in dom.select("#blogColl .f_link_b")
if checkBlog(_["href"])]

['http://adam24eve.tistory.com/858',
 'http://cscs0717.tistory.com/27',
 'http://listup.tistory.com/248',
 'http://34344541.tistory.com/43']

In [115]:
def parseURL(seed):
    html = download("get", seed)
    dom = BeautifulSoup(html.text, 'lxml')
    
    return [requests.compat.urljoin(seed, _["href"])
     for _ in dom.find_all("a") if _.has_attr("href") and len(_["href"]) > 3
           and checkBlog(requests.compat.urljoin(seed, _["href"]))]

In [119]:
queue = list()
queue.extend([_["href"] for _ in dom.select("#blogColl .f_link_b")
             if checkBlog(_["href"])])
seen = list()

while queue:
    baseURL = queue.pop(0) # 무조건 앞에 있는 게 나온다. 선입선출. pop(-1) 로 바꾸면 스택이 된다.
    seen.append(baseURL)
    time.sleep(1) # 슬립
    linkList = parseURL(baseURL) # 중복되는 URL이 생긴다.
    for link in linkList: # 중복되는 url 있으면 피할 것이다.
        if link not in queue and link not in seen:
            queue.append(link)
    print("Queue: {0}, Seen : {1}".format(len(queue), len(seen)))
#     queue.extend(linkList)

Queue: 44, Seen : 1
Queue: 60, Seen : 2
Queue: 83, Seen : 3
Queue: 90, Seen : 4
Queue: 96, Seen : 5
Queue: 102, Seen : 6
Queue: 108, Seen : 7
Queue: 111, Seen : 8
Queue: 118, Seen : 9
Queue: 121, Seen : 10
Queue: 120, Seen : 11
Queue: 119, Seen : 12
Queue: 118, Seen : 13
Queue: 117, Seen : 14
Queue: 116, Seen : 15
Queue: 115, Seen : 16
Queue: 116, Seen : 17
Queue: 115, Seen : 18
Queue: 114, Seen : 19
Queue: 113, Seen : 20
Queue: 128, Seen : 21
Queue: 140, Seen : 22
Queue: 148, Seen : 23
Queue: 150, Seen : 24
Queue: 150, Seen : 25
Queue: 157, Seen : 26
Queue: 166, Seen : 27
Queue: 171, Seen : 28
Queue: 170, Seen : 29
Queue: 170, Seen : 30
Queue: 180, Seen : 31
Queue: 180, Seen : 32
Queue: 185, Seen : 33
Queue: 185, Seen : 34
Queue: 193, Seen : 35
Queue: 203, Seen : 36
Queue: 209, Seen : 37
Queue: 213, Seen : 38
Queue: 233, Seen : 39
Queue: 248, Seen : 40
Queue: 265, Seen : 41
Queue: 276, Seen : 42
Queue: 290, Seen : 43
Queue: 294, Seen : 44
Queue: 295, Seen : 45
Queue: 294, Seen : 46
Qu

KeyboardInterrupt: 

------------------------------------------------------

In [122]:
def parseURL(seed):
    html = download("get", seed[0])
    dom = BeautifulSoup(html.text, 'lxml')
    
    return [(requests.compat.urljoin(seed[0], _["href"]), seed[1]+1)
     for _ in dom.find_all("a") if seed[1] < 2 and _.has_attr("href") and len(_["href"]) > 3
           and checkBlog(requests.compat.urljoin(seed[0], _["href"]))]

In [123]:
queue = list()
queue.extend([(_["href"], 1) for _ in dom.select("#blogColl .f_link_b")
             if checkBlog(_["href"])])
seen = list()

while queue:
    baseURL = queue.pop(0) 
    seen.append(baseURL)
    time.sleep(1) # 슬립
    linkList = parseURL(baseURL) # 중복되는 URL이 생긴다.
    for link in linkList: # 중복되는 url 있으면 피할 것이다.
        if link not in queue and link not in seen:
            queue.append(link)
    print("Queue: {0}, Seen : {1}".format(len(queue), len(seen)))
#     queue.extend(linkList)

Queue: 45, Seen : 1
Queue: 61, Seen : 2
Queue: 85, Seen : 3
Queue: 92, Seen : 4
Queue: 91, Seen : 5
Queue: 90, Seen : 6
Queue: 89, Seen : 7
Queue: 88, Seen : 8
Queue: 87, Seen : 9
Queue: 86, Seen : 10
Queue: 85, Seen : 11
Queue: 84, Seen : 12
Queue: 83, Seen : 13
Queue: 82, Seen : 14
Queue: 81, Seen : 15
Queue: 80, Seen : 16
Queue: 79, Seen : 17
Queue: 78, Seen : 18
Queue: 77, Seen : 19
Queue: 76, Seen : 20
Queue: 75, Seen : 21
Queue: 74, Seen : 22
Queue: 73, Seen : 23
Queue: 72, Seen : 24
Queue: 71, Seen : 25
Queue: 70, Seen : 26
Queue: 69, Seen : 27
Queue: 68, Seen : 28
Queue: 67, Seen : 29
Queue: 66, Seen : 30
Queue: 65, Seen : 31
Queue: 64, Seen : 32
Queue: 63, Seen : 33
Queue: 62, Seen : 34
Queue: 61, Seen : 35
Queue: 60, Seen : 36
Queue: 59, Seen : 37
Queue: 58, Seen : 38
Queue: 57, Seen : 39
Queue: 56, Seen : 40
Queue: 55, Seen : 41
Queue: 54, Seen : 42
Queue: 53, Seen : 43
Queue: 52, Seen : 44
Queue: 51, Seen : 45
Queue: 50, Seen : 46
Queue: 49, Seen : 47
Queue: 48, Seen : 48
Q

In [124]:
seen

[('http://adam24eve.tistory.com/858', 1),
 ('http://cscs0717.tistory.com/27', 1),
 ('http://listup.tistory.com/248', 1),
 ('http://34344541.tistory.com/43', 1),
 ('http://adam24eve.tistory.com/category', 2),
 ('http://adam24eve.tistory.com/category/Cosmos', 2),
 ('http://adam24eve.tistory.com/category/Mystery%20world', 2),
 ('http://adam24eve.tistory.com/category/Star', 2),
 ('http://adam24eve.tistory.com/category/Political%20', 2),
 ('http://adam24eve.tistory.com/category/It-game', 2),
 ('http://adam24eve.tistory.com/guestbook', 2),
 ('https://adam24eve.tistory.com/rss', 2),
 ('http://adam24eve.tistory.com/858', 2),
 ('http://adam24eve.tistory.com/tag/%EA%B9%80%EC%98%81%EA%B4%91', 2),
 ('http://adam24eve.tistory.com/tag/%EA%B9%80%EC%98%81%EA%B4%91%20%EB%82%98%EC%9D%B4',
  2),
 ('http://adam24eve.tistory.com/tag/%EA%B9%80%EC%98%81%EA%B4%91%20%EB%B0%95%EB%B3%B4%EC%98%81%20%EC%97%B4%EC%95%A0',
  2),
 ('http://adam24eve.tistory.com/tag/%EA%B9%80%EC%98%81%EA%B4%91%20%ED%82%A4',
  2),
 ('ht

***************************************

## SQLite 연계

- primary key 중요!(pk 값에 절대 문자열 쓰면 안된다.)
- table 하나에 최소 (id, url, seen, date) 는 필요하다.

- table1 = id, path, param, seen, date
- table2 = id, netloc, date
- url = netloc + path + param

In [210]:
import sqlite3

con = sqlite3.connect("bot.db")
cur = con.cursor()

In [211]:
# 실제 데이터는 path , param 만 입력
cur.executescript("""

    DROP TABLE IF EXISTS table1;
    CREATE TABLE table1(
    id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
    table2_id INTEGER NOT NULL,
    path TEXT NOT NULL,
    param TEXT,
    depth INTEGER NOT NULL,
    inbound INTEGER NOT NULL,
    seen BOOLEAN DEFAULT FALSE NOT NULL,
    date TIMESTAMP DEFAULT CURRENT_TIMESTAMP NOT NULL
    );
    
    DROP TABLE IF EXISTS table2;
    CREATE TABLE table2(
    id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
    netloc TEXT NOT NULL,
    date TIMESTAMP DEFAULT CURRENT_TIMESTAMP NOT NULL
    );
""")
con.commit()

In [207]:
# Seed URLs --> DB Insert
url = "https://www.google.com/search"
html = download("get", url, param= {"q":"박보영"})
dom = BeautifulSoup(html.text, "lxml")

for href in [_.find_parent()["href"] for _ in dom.select(".LC20lb")]:
    _urlparse = requests.compat.urlparse(href)
    netloc = "://".join(requests.compat.urlparse(href)[:2])
    cur.execute("SELECT id FROM table2 WHERE netloc=? LIMIT 0,1", [netloc])
    netlocID = cur.fetchone()
    if not netlocID:
        cur.execute("INSERT INTO table2(netloc) VALUES(?)", [netloc])
        
        con.commit()
        
        cur.execute("SELECT id FROM table2 WHERE netloc=? LIMIT 0,1", [netloc])
        
        netlocID = cur.fetchone()
        
    cur.execute("""
        INSERT INTO table1(table2_id, path, param, depth, inbound) VALUES(?, ?, ?, ?, ?)
        """, [netlocID[0], _urlparse[2], _urlparse[4], 1, 0]

    con.commit()
    
    
# DB Select Limit 0,1 해서 못가져오면 다 한거다.

SyntaxError: invalid syntax (<ipython-input-207-4d28c2fcf62f>, line 24)

In [209]:
cur.close()
con.close()

In [141]:
requests.compat.urlparse(html.request.url)

ParseResult(scheme='https', netloc='www.google.com', path='/search', params='', query='q=%EB%B0%95%EB%B3%B4%EC%98%81', fragment='')

In [206]:
while True:
    cur.execute("""
                SELECT table1.id, table2.netloc, table1.path, table1.param
                FROM table1
                JOIN table2
                    ON table1.table2_id = table2.id
                WHERE table1.seen=FALSE
                ORDER BY table1.date ASC
                LIMIT 0 ,1;
            """)
    seed = cur.fetchone()
    if not seed:
        break;
        
    print(seed)
    print("{0}{1}?{2}".format(seed[1], seed[2], seed[3]))
    break

-------------------------------

In [202]:
def parseURL(seed):
    html = download("get", seed)
    dom = BeautifulSoup(html.text, 'lxml')
    
    return [requests.compat.urljoin(seed, _["href"]) for _ in dom.find_all("a") if _.has_attr("href") and len(_["href"]) > 3]

In [203]:
i = 0
while True:
    cur.execute("""
                SELECT table1.id, table2.netloc, table1.path, table1.param, table1.depth, table2.id
                FROM table1
                JOIN table2
                    ON table1.table2_id = table2.id
                WHERE table1.seen=FALSE and table1.depth < 3
                ORDER BY table1.date ASC
                LIMIT 0 ,1;
            """)
    seed = cur.fetchone()
    if not seed or i > 1000:
        break;
    i += 1
        
    cur.execute("""
        UPDATE table1
        SET seen=TRUE
        WHERE id=?
    """, [seed[0]])
    con.commit()
    
    baseURL = "{0}{1}?{2}".format(seed[1], seed[2], seed[3])
    for href in parseURL(baseURL):
        _urlparse = requests.compat.urlparse(href)
        netloc = "://".join(_urlparse[:2])
        cur.execute("SELECT id FROM table2 WHERE netloc=? LIMIT 0,1", [netloc])
        netlocID = cur.fetchone()
        if not netlocID:
            cur.execute("INSERT INTO table2(netloc) VALUES(?)", [netloc])

            con.commit()

            cur.execute("SELECT id FROM table2 WHERE netloc=? LIMIT 0,1", [netloc])

            netlocID = cur.fetchone()
        
    cur.execute("INSERT INTO table1(table2_id, path, param, depth, inbound) VALUES(?, ?, ?, ?, ?)",\
                [netlocID[0], _urlparse[2], _urlparse[4], seed[4]+1, seed[5]])
    con.commit()
    print(cur.lastrowid, netlocID)

In [204]:
cur.execute("""
    select inbound, count(inbound)
    from table1
    where table2_id=10 group by inbound;
""")
cur.fetchall()

[]

In [205]:
cur.execute("""
    select inbound, count(inbound)
    from table1
    where inbound=1;
""")
cur.fetchall()

[(None, 0)]